In [71]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torchmetrics.classification import Accuracy
import torchmetrics

import torchvision.models as models

device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!pip install torchmetrics

In [5]:
import zipfile
with zipfile.ZipFile("tomato_data.zip", 'r') as zip:
  zip.extractall("tomato_data")

In [6]:
path= '/content/tomato_data'

In [73]:
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),  #Removed RandomCrop(32) as it resized the image back
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                     [0.229, 0.224, 0.225])   #pretrained ImageNet values
])

test_transform= transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                     [0.229, 0.224, 0.225])
])

In [74]:
train_path= '/content/tomato_data/data/Training_set'
test_path= '/content/tomato_data/data/Testing_set'

train_data= datasets.ImageFolder(root=train_path, transform= train_transform)
test_set= datasets.ImageFolder(root=test_path, transform= test_transform)

for (img, label) in train_data:
  print(img.shape)
  break

torch.Size([3, 128, 128])


In [75]:
train_size = int(0.85 * len(train_data))
val_size   = len(train_data) - train_size

train_set, val_set = random_split(train_data, [train_size, val_size], generator=torch.Generator().manual_seed(42))


In [76]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_set,  batch_size=64, shuffle=False)


for (img, label) in train_loader:
  print(img.shape)
  break

torch.Size([64, 3, 128, 128])


In [93]:
model= models.resnet50(pretrained=True).to(device)

num_classes= 10
model.fc= nn.Linear(model.fc.in_features, num_classes).to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [94]:

loss_function= nn.CrossEntropyLoss()

In [95]:
optimizer= torch.optim.Adam(model.parameters(), lr= 0.001)

scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= "max", factor= 0.5, patience= 3)


In [96]:
def train(dataloader, model, loss_function, optimizer):
  model.train()
  total_loss= 0

  for batch, (image, label) in enumerate(dataloader):
    image= image.to(device)
    label= label.to(device)

    prediction= model(image)
    loss= loss_function(prediction, label)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss+= loss

  avg_loss= total_loss / len(dataloader)
  print(f"Training Average Loss: {avg_loss:.4f}")

In [97]:
val_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def validate(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    val_accuracy.reset()

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            val_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = val_accuracy.compute() * 100  # Convert to %

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")

    return accuracy


In [98]:
test_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def test(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    test_accuracy.reset()   #Reset metric state for each epoch

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            test_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = test_accuracy.compute() * 100  # Convert to %

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

    return accuracy

In [99]:
epochs = 90
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train(train_loader, model, loss_function, optimizer)
    val_acc = validate(val_loader, model, loss_function)
    scheduler.step(val_acc)
print("\n")
print("-----------------------------------------------------------------")
print("\n")
test(test_loader, model, loss_function)


Epoch 1/90
Training Average Loss: 1.5724
Validation Loss: 1.8984, Validation Accuracy: 49.93%

Epoch 2/90
Training Average Loss: 1.3544
Validation Loss: 1.5319, Validation Accuracy: 51.63%

Epoch 3/90
Training Average Loss: 1.3262
Validation Loss: 3.5468, Validation Accuracy: 32.64%

Epoch 4/90
Training Average Loss: 1.3815
Validation Loss: 1.7668, Validation Accuracy: 48.63%

Epoch 5/90
Training Average Loss: 1.3049
Validation Loss: 1.5368, Validation Accuracy: 43.43%

Epoch 6/90
Training Average Loss: 1.2158
Validation Loss: 1.6874, Validation Accuracy: 47.20%

Epoch 7/90
Training Average Loss: 1.0699
Validation Loss: 1.2770, Validation Accuracy: 56.05%

Epoch 8/90
Training Average Loss: 1.1122
Validation Loss: 1.3704, Validation Accuracy: 53.32%

Epoch 9/90
Training Average Loss: 1.0023
Validation Loss: 1.2838, Validation Accuracy: 55.79%

Epoch 10/90
Training Average Loss: 0.9614
Validation Loss: 1.1726, Validation Accuracy: 55.92%

Epoch 11/90
Training Average Loss: 0.9916
Valida

tensor(69.8824, device='cuda:0')